In [1]:
!pip install pulp

In [2]:
import pulp
import math

In [3]:
# Updated Drones Data with Quantity
drones = [
    {'name': 'MQ-9 Reaper', 'endurance': 14, 'coverage': 2, 'speed': 300, 'quantity': 2},
    {'name': 'MQ-1 Predator', 'endurance': 24, 'coverage': 1.5, 'speed': 135, 'quantity': 3},
    {'name': 'MQ-8 Fire Scout', 'endurance': 8, 'coverage': 1, 'speed': 132, 'quantity': 1},
    {'name': 'RQ-4 Global Hawk', 'endurance': 32, 'coverage': 5, 'speed': 391, 'quantity': 1},
    {'name': 'RQ-11 Raven', 'endurance': 1.5, 'coverage': 0.05, 'speed': 18.64, 'quantity': 1},
    {'name': 'RQ-20 Puma', 'endurance': 2, 'coverage': 0.1, 'speed': 52, 'quantity': 1},
    {'name': 'RQ-21 Blackjack', 'endurance': 16, 'coverage': 1, 'speed': 100, 'quantity': 1},
    {'name': 'TAI Aksungur', 'endurance': 50, 'coverage': 2, 'speed': 50, 'quantity': 1},
    {'name': 'Chengdu Wing Loong II', 'endurance': 10, 'coverage': 2, 'speed': 100, 'quantity': 1},
    {'name': 'Bayraktar TB2', 'endurance': 24, 'coverage': 1.5, 'speed': 80, 'quantity': 1},
]

points_data = [
    {'lat': 34.0522, 'lng': -118.2437, 'order': 1, 'size': 150, 'time': 5},
    {'lat': 36.7783, 'lng': -119.4179, 'order': 2, 'size': 250, 'time': 3},
]

# User input for minimum coverage percentage
min_coverage_percent = 90

### Drones: 
A list of drones, where each drone 

$endurance_{i}$: The maximum duration (in hours) the drone can operate without refueling or recharging.

$coverage_{i}$: The width of the area (in km) the drone can cover in a single pass at a given altitude.

$speed_{i}$: The speed of the drone (in km/h).

$quantity_{i}$: The number of available units for each drone type.

### Points Data: 
A list of points of interest, where each point 

$size_{j}$: The area (in km²) that needs to be covered or surveyed.

$time_{j}$: The duration (in hours) for which the coverage needs to be maintained.

### Min Coverage Percent: 
The minimum percentage of the point's area that needs to be covered, represented as 
min_coverage_percent

### Decision Variables:
x_ij: Binary variable that equals 1 if drone i is used to cover point j, and 0 otherwise.


### Objective Function:
Minimize the total number of drones used across all points:
$ min\sum \limits _{i=1} ^n \sum \limits _{j=1} ^m x_{ij} $


### Constraints:
#### Coverage Constraint: 
Ensure that the drones assigned to each point j collectively cover at least the required area size of the point.

$ \sum \limits _{i=1} ^n x_{ij} * coverage_{i} * speed_{i} \ge size_{j}, $ for all $j$

#### Endurance Constraint: 
Ensure that each drone assigned to a point can cover it for the required duration.

$ x_{ij}  *  time_{j} $ $ \le endurance_{i}$, for all $i$ and $j$

#### Assignment Constraint: 
A drone can be assigned to a point only if it's available (considering the quantity available for each drone type).

$ x_{ij} \le quantity_{i}$, for all $i$ and $j$

In [4]:
# Haversine formula to calculate the great-circle distance between two points
def calculate_distance(point1, point2):
    R = 6371  # Earth radius in km
    lat1, lon1 = map(math.radians, [point1['lat'], point1['lng']])
    lat2, lon2 = map(math.radians, [point2['lat'], point2['lng']])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Calculate distances between consecutive points
for i in range(len(points_data) - 1):
    points_data[i]['distance_to_next'] = calculate_distance(points_data[i], points_data[i + 1])
points_data[-1]['distance_to_next'] = 0  # Last point has no next point

In [5]:
# Define the problem as a minimization problem
problem = pulp.LpProblem("DroneDeploymentMultiplePoints", pulp.LpMinimize)

# Decision variables: x_{i}_j = 1 if drone i is used for point j, 0 otherwise
drone_vars = pulp.LpVariable.dicts("drone", [(i, j) for i in range(len(drones)) for j in range(len(points_data))], cat=pulp.LpBinary)

# Objective Function: Minimize the number of drones used
problem += pulp.lpSum(drone_vars[i, j] for i in range(len(drones)) for j in range(len(points_data))), "MinimizeDrones"

# Constraints
for j, point in enumerate(points_data):
    # Ensure the coverage for each point is met
    problem += pulp.lpSum(drone_vars[i, j] * drones[i]['coverage'] * drones[i]['speed'] for i in range(len(drones))) >= point['size'], f"Coverage_Point_{j}"
    
    # Ensure that each drone assigned can cover the point for the required time
    for i, drone in enumerate(drones):
        problem += drone_vars[i, j] * point['time'] <= drones[i]['endurance'], f"Endurance_Drone_{i}_Point_{j}"

# Solve the problem
problem.solve()

# Output the results
print("Status:", pulp.LpStatus[problem.status])

if pulp.LpStatus[problem.status] == "Optimal":
    drone_usage = {drone['name']: 0 for drone in drones}  # Initialize a dictionary to count the usage of each drone type

    for i, drone in enumerate(drones):
        for j in range(len(points_data)):
            if pulp.value(drone_vars[i, j]) == 1:
                print(f"Drone {drone['name']} is selected to cover Point {points_data[j]['order']}.")
                drone_usage[drone['name']] += 1  # Increment the count for the drone

    # Display the total number of each drone type used
    for drone_name, count in drone_usage.items():
        print(f"Total number of {drone_name} drones used: {count}")
else:
    print("No feasible solution found. One or more points cannot be covered by the available drones.")


Status: Optimal
Drone MQ-9 Reaper is selected to cover Point 2.
Drone Chengdu Wing Loong II is selected to cover Point 1.
Total number of MQ-9 Reaper drones used: 1
Total number of MQ-1 Predator drones used: 0
Total number of MQ-8 Fire Scout drones used: 0
Total number of RQ-4 Global Hawk drones used: 0
Total number of RQ-11 Raven drones used: 0
Total number of RQ-20 Puma drones used: 0
Total number of RQ-21 Blackjack drones used: 0
Total number of TAI Aksungur drones used: 0
Total number of Chengdu Wing Loong II drones used: 1
Total number of Bayraktar TB2 drones used: 0


In [6]:
problem

DroneDeploymentMultiplePoints:
MINIMIZE
1*drone_(0,_0) + 1*drone_(0,_1) + 1*drone_(1,_0) + 1*drone_(1,_1) + 1*drone_(2,_0) + 1*drone_(2,_1) + 1*drone_(3,_0) + 1*drone_(3,_1) + 1*drone_(4,_0) + 1*drone_(4,_1) + 1*drone_(5,_0) + 1*drone_(5,_1) + 1*drone_(6,_0) + 1*drone_(6,_1) + 1*drone_(7,_0) + 1*drone_(7,_1) + 1*drone_(8,_0) + 1*drone_(8,_1) + 1*drone_(9,_0) + 1*drone_(9,_1) + 0
SUBJECT TO
Coverage_Point_0: 600 drone_(0,_0) + 202.5 drone_(1,_0) + 132 drone_(2,_0)
 + 1955 drone_(3,_0) + 0.932 drone_(4,_0) + 5.2 drone_(5,_0)
 + 100 drone_(6,_0) + 100 drone_(7,_0) + 200 drone_(8,_0) + 120 drone_(9,_0)
 >= 150

Endurance_Drone_0_Point_0: 5 drone_(0,_0) <= 14

Endurance_Drone_1_Point_0: 5 drone_(1,_0) <= 24

Endurance_Drone_2_Point_0: 5 drone_(2,_0) <= 8

Endurance_Drone_3_Point_0: 5 drone_(3,_0) <= 32

Endurance_Drone_4_Point_0: 5 drone_(4,_0) <= 1.5

Endurance_Drone_5_Point_0: 5 drone_(5,_0) <= 2

Endurance_Drone_6_Point_0: 5 drone_(6,_0) <= 16

Endurance_Drone_7_Point_0: 5 drone_(7,_0) <